<div class="alert alert-block alert-info">
⚠️ <b>Note:</b> We recommend using <code>Python ≥ 3.10</code> for full compatibility with <code>torch-geometric</code>.
</div>

In [ ]:
import pandas as pd
import torch
from torch_geometric.data import TemporalData

In [ ]:
#Load the dataset
dataset = pd.read_csv('./data/edge_events.csv')

In [ ]:
# Extract unique nodes and create mapping dictionaries
nodes = pd.unique(dataset[["src", "dst"]].to_numpy().ravel())
node2idx = {n: i for i, n in enumerate(nodes)}
idx2node = {i: n for n, i in node2idx.items()}

# Map source and destination nodes to integer IDs
src = dataset["src"].map(node2idx).to_numpy()
dst = dataset["dst"].map(node2idx).to_numpy()

# Extract timestamps and labels
t = dataset["timestamp"].astype(int).to_numpy()
msg = pd.get_dummies(dataset["label"]).to_numpy()

# Build TemporalData object
data = TemporalData(
    src=torch.from_numpy(src).long(),
    dst=torch.from_numpy(dst).long(),
    t=torch.from_numpy(t).long(),
    msg=torch.from_numpy(msg).float(),
)

In [ ]:

import torch
import torch.nn as nn
from torch_geometric.nn import TemporalConv

class SimpleTGNN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SimpleTGNN, self).__init__()
        self.temporal_conv = TemporalConv(in_channels, out_channels, kernel_size=3)

    def forward(self, data):
        x, edge_index, t = data.x, data.edge_index, data.t
        x = self.temporal_conv(x, edge_index, t)
        return x

# Define model
model = SimpleTGNN(in_channels=10, out_channels=5)